In [1]:
#!pip install transformers

#imports
import numpy as np
import pandas as pd
# import torch
import tensorflow as tf
from sklearn.model_selection import train_test_split

#transformers import
from transformers import DistilBertTokenizerFast, DistilBertModel
from transformers import TFDistilBertForSequenceClassification, TFTrainer, TFTrainingArguments

ModuleNotFoundError: No module named 'tensorflow'